# Predictive Analysis on Employee Network Missing Data

Libraries:
- NetworkX, Scikit-learn, Pandas, & Numpy

1. Graphs read in using Network
2. Graph types determined using various graph features (distance measurements, centrality, etc.) among number of potential choices (Barabasi-Albert preferential attachment graphs, Watts-Strogatz small world network models, etc.)
3. Goal: identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary
    -  Feature extraction performed using graph **node-wise** features (betweenness centrality, degree centrality, closeness centraliity, eigenvector centrality, page rank, HITS algorithm hub and authority scores)
    - Training and test data prepared by segmenting feature values by presence of target variable
    - Random Forest Classifier fit on available data to produce model
    - Model metrics determined through Coursera autograder and ROC-curve AUC of > 0.85 achieved
4. Goal: using partially missing data on upcoming employee connections, predict future connections between all employees of the network.
    - Feature extraction on **edge-wise** features (common neighbor coefficient, resource allocation index, jaccard coefficient, preferential attachment score, common neighbor soundarajan hopcroft score)
    - Data split into training and test sets for prediction on missing data
    - Gradient Boosting Classifier fit on non-missing data to create model
    - Model run against witheld missing data on Coursera autograder and ROC-curve AUC > 0.85 achieved.

In [2]:
import pickle
import networkx as nx
import numpy as np
import pandas as pd

In [3]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

---

## Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [4]:
G1 = nx.read_gpickle("assets/A4_P1_G1")
G2 = nx.read_gpickle("assets/A4_P1_G2")
G3 = nx.read_gpickle("assets/A4_P1_G3")
G4 = nx.read_gpickle("assets/A4_P1_G4")
G5 = nx.read_gpickle("assets/A4_P1_G5")
P1_Graphs = [G1, G2, G3, G4, G5]

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs using any methodology and determine which of the three algorithms generated each graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [5]:
sorted(list(nx.preferential_attachment(G4)), key=lambda x: x[2], reverse=True)[:3]

[(0, 11, 1404), (0, 9, 1170), (9, 11, 1080)]

In [6]:
def graph_identification():
    # YOUR CODE HERE
    status = [1, 1, 1, 1, 1]
    for i, g in enumerate(P1_Graphs):
        if max(list(nx.preferential_attachment(g)), key=lambda x: x[2])[2] > 1000:
            status[i] = "PA"
        elif nx.average_shortest_path_length(g) < 10:
            status[i] = "SW_H"
        else:
            status[i] = "SW_L"

    return status

In [10]:
ans_one = graph_identification()
assert type(ans_one) == list, "You must return a list"

---

## Company Emails

For the second part of this assignment you will be working with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [12]:
G = pickle.load(open("assets/email_prediction_NEW.txt", "rb"))

print(f"Graph with {len(nx.nodes(G))} nodes and {len(nx.edges(G))} edges")

Graph with 1005 nodes and 16706 edges


In [13]:
G_dict = sorted(dict(G.nodes(data=True)).items(), key=lambda x: x[0])
G_idx = [g[0] for g in G_dict]

In [14]:
G_data = [g[1] for g in G_dict]
nodeData = pd.DataFrame(G_data)

In [16]:
nodeData.head(3)

,Department,ManagementSalary
0,1,0.0
1,1,NaN
2,21,NaN


In [346]:
trueSal = pd.read_csv('assets/true_node_salaries', header=None).set_index(0).sort_index()

In [209]:
(pd.DataFrame(dict(G.nodes(data=True))
                    .values())
                    .ManagementSalary
                    .value_counts(dropna=False))

0.0    634
NaN    252
1.0    119
Name: ManagementSalary, dtype: int64

---
### Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features of your choice using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will recieve full points.

Using your trained classifier, return a Pandas series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [17]:
list(G.nodes(data=True))[:3]  # print the first 3 nodes

[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (581, {'Department': 3, 'ManagementSalary': 0.0})]

In [22]:
def salary_predictions():
    
    G_dict = sorted(dict(G.nodes(data=True)).items(), key=lambda x: x[0])
    
    G_data = [g[1] for g in G_dict]
    nodeData = pd.DataFrame(G_data)
    
    clusteringScores = pd.Series([c[1] for c in sorted(nx.clustering(G).items(), key=lambda x: x[0])])
    hubScores = pd.Series([h[1] for h in sorted(nx.hits(G)[0].items(), key=lambda x: x[0])])
    
    btwnCntrl = pd.Series([c[1] for c in sorted(nx.betweenness_centrality(G).items(), key=lambda x: x[0])])
    closenessCntrl = pd.Series([c[1] for c in sorted(nx.closeness_centrality(G).items(), key=lambda x: x[0])])
    degCntrl = pd.Series([c[1] for c in sorted(nx.degree_centrality(G).items(), key=lambda x: x[0])])
    eigenCntrl = pd.Series([c[1] for c in sorted(nx.eigenvector_centrality(G).items(), key=lambda x: x[0])])
    pageRank = pd.Series([c[1] for c in sorted(nx.pagerank(G).items(), key=lambda x: x[0])])

    nameMap = {0:'ClusteringScore', 1:'HubScore', 2:'BetweennessCentrality', 3:'ClosenessCentrality', 4:'DegreeCentrality', 5:'EigenvectorCentrality', 6:'PageRank'}

    nodeScores = pd.concat([nodeData, clusteringScores, hubScores, btwnCntrl, closenessCntrl, degCntrl, eigenCntrl, pageRank], axis=1)\
    .rename(nameMap, axis=1)
    
    train = nodeScores[~nodeScores.ManagementSalary.isna()]
    test = nodeScores[nodeScores.ManagementSalary.isna()]

    X_train = train.drop('ManagementSalary', axis=1)
    y_train = train.ManagementSalary

    X_test = test.drop('ManagementSalary', axis=1)
    y_test = test.ManagementSalary

    randClf = RandomForestClassifier(n_jobs=-1, random_state=42)
    randClf.fit(X_train, y_train)

    pred = pd.Series(randClf.predict_proba(X_test)[:,0], X_test.index)
    
    return pred
    
    # YOUR CODE HERE

In [23]:
salary_predictions()

/Users/rohilk/opt/anaconda3/lib/python3.9/site-packages/networkx/algorithms/link_analysis/hits_alg.py:78: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G, nodelist=list(G), dtype=float)


1       0.91
2       0.05
5       0.01
8       0.93
14      0.94
        ... 
992     1.00
994     1.00
996     1.00
1000    1.00
1001    0.99
Length: 252, dtype: float64

In [28]:
ans_salary_preds = salary_predictions()
assert (
    type(ans_salary_preds) == pd.core.series.Series
), "You must return a Pandas series"
assert len(ans_salary_preds) == 252, "The series must be of length 252"

/Users/rohilk/opt/anaconda3/lib/python3.9/site-packages/networkx/algorithms/link_analysis/hits_alg.py:78: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G, nodelist=list(G), dtype=float)


---
### New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [29]:
future_connections = pd.read_csv(
    "assets/Future_Connections.csv", index_col=0, converters={0: eval}
)
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


In [30]:
future_connections['Future Connection'].value_counts(dropna=False)

Future Connection
0.0    337002
NaN    122112
1.0     29332
Name: count, dtype: int64

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to:      
1. Create a matrix of features of your choice for the edges found in `future_connections` using Networkx     
2. Train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data     
3. Predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will recieve full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [32]:
future = future_connections.sort_index().reset_index()

In [33]:
future.head(3)

,index,Future Connection
0,"(0, 2)",0.0
1,"(0, 3)",0.0
2,"(0, 4)",0.0


In [38]:
def new_connections_predictions():

    # YOUR CODE HERE
    commNeighbor = pd.Series([c[2] for c in [*nx.common_neighbor_centrality(G)]])
    rsrcAlloc = pd.Series([c[2] for c in [*nx.resource_allocation_index(G)]])
    jaccardCoef = pd.Series([c[2] for c in [*nx.jaccard_coefficient(G)]])
    prefAttach = pd.Series([c[2] for c in [*nx.preferential_attachment(G)]])
    cnSoundarajan = pd.Series([c[2] for c in [*nx.cn_soundarajan_hopcroft(G, community='Department')]])

    colMap = {0:'CommonNeighborCentrality', 1:'ResourceAllocationIndex', 2:'JaccardCoefficient', 3:'PreferentialAttachment', 4:'CNSoundarajanHopcroft'}
    
    edgeScores = pd.concat([future, commNeighbor, rsrcAlloc, jaccardCoef, prefAttach, cnSoundarajan],axis=1).rename(colMap, axis=1).set_index('index')
    
    train = edgeScores[~edgeScores['Future Connection'].isna()]
    test = edgeScores[edgeScores['Future Connection'].isna()]
    
    X_train = train.drop('Future Connection', axis=1)
    y_train = train['Future Connection']
    
    X_test = test.drop('Future Connection', axis=1)
    y_test = test['Future Connection']
    
    randClf = GradientBoostingClassifier(random_state=42)
    randClf.fit(X_train, y_train)
    
    pred = pd.Series(randClf.predict_proba(X_test)[:,0], X_test.index)
    
    return pred

In [39]:
new_connections_predictions()

index
(0, 9)          0.968270
(0, 19)         0.931657
(0, 20)         0.761932
(0, 35)         0.983714
(0, 38)         0.985642
                  ...   
(998, 999)      0.986609
(1000, 1002)    0.987174
(1000, 1003)    0.987174
(1000, 1004)    0.986745
(1001, 1002)    0.987174
Length: 122112, dtype: float64

In [326]:
ans_prob_preds = new_connections_predictions()
assert type(ans_prob_preds) == pd.core.series.Series, "You must return a Pandas series"
assert len(ans_prob_preds) == 122112, "The series must be of length 122112"